# Sistemas Biometricos / Biometric Systems
## ECG Biometrics - Scikitlearn
## 15/10/2024


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import wfdb

# Database from Physionet
https://physionet.org/content/ecgiddb/1.0.0/

In [5]:
signals1, fields1 = wfdb.rdsamp('ecg-id-database-1.0.0/Person_01/rec_1')
signals2, fields1 = wfdb.rdsamp('ecg-id-database-1.0.0/Person_02/rec_1')
signals3, fields1 = wfdb.rdsamp('ecg-id-database-1.0.0/Person_03/rec_1')

In [6]:
from biosppy.signals import ecg
out1 = ecg.ecg(signal=signals1[:,1], sampling_rate=500., show=False)
out2 = ecg.ecg(signal=signals2[:,1], sampling_rate=500., show=False)
out3 = ecg.ecg(signal=signals3[:,1], sampling_rate=500., show=False)

In [35]:
out1['templates'].shape

(40, 600)

In [37]:
templates = np.vstack((out1['templates'], out2['templates'], out3['templates']))
templates.shape

(129, 600)

In [60]:
labels = np.hstack( (np.ones(len(out1['templates']))*1, np.ones(len(out2['templates']))*2, np.ones(len(out3['templates']))*3))
labels

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])

In [81]:
templates.shape

(129, 600)

In [102]:
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)

In [105]:
clf.fit(templates[:-1,:],labels[:-1])

SVC(C=100.0, gamma=0.001)

In [114]:
templates[-1,:].shape

(600,)

In [113]:
np.array([templates[-1,:]]).shape

(1, 600)

In [110]:
clf.predict([templates[-1,:]])

array([3.])

# Authentication

In [115]:
user1Labels = np.hstack( (np.ones(len(out1['templates']))*1, np.ones(len(out2['templates']))*0, np.ones(len(out3['templates']))*0))

In [145]:
len(out1['templates'])

40

In [117]:
clfUser1 = svm.SVC(gamma=0.001, C=100.)
clfUser1.fit(templates[:-1,:],user1Labels[:-1])

SVC(C=100.0, gamma=0.001)

In [118]:
clfUser1.predict([templates[-1,:]])

array([0.])

In [149]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(templates, user1Labels, test_size=0.4, random_state=0)

In [154]:
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
print(clf.score(X_test, y_test))
y_predict = clf.predict(X_test)
confusion_matrix(y_test, y_predict)


1.0


array([[36,  0],
       [ 0, 16]])

# Cross Valitation

In [156]:
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

kf = KFold(n_splits=5, shuffle=True)
for train, test in kf.split(templates):
    #print("%s %s" % (train, test))
    train == 
    clfUser1.fit(templates[train,:],user1Labels[train])
    y_pred =clfUser1.predict(templates[test])
    #print(user1Labels[test])
    # print(y_pred)
    c= confusion_matrix(user1Labels[test], y_pred)
    print(c)

[[20  0]
 [ 6  0]]
[[21  0]
 [ 5  0]]
[[17  0]
 [ 9  0]]
[[17  0]
 [ 9  0]]
[[14  0]
 [11  0]]


In [166]:
print("number of samples user 1:,",sum(user1Labels[train] == 1), "remaining sampples",sum(user1Labels[train] == 0))

number of samples user 1:, 29 remaining sampples 75


In [147]:
from sklearn.model_selection import cross_val_score
clfUser1 = svm.SVC(gamma=0.001, C=100.)

print(cross_val_score(clfUser1, templates, user1Labels, cv=))

[0.69230769 0.69230769 0.69230769 0.69230769 0.68      ]
